In [16]:
import pandas as pd
import numpy as np 
from simulate import generateData

In [17]:
# number of observations
n=4000

# number of (Gaussian) covariates
p=20

# signal strength
signal_strength = 5

# Parameters for Distribution to draw Betas from. Betas ~ N(mu, stdev^2)
mu = 0
stdev = 1.0

X, Beta, y, means = generateData(dist = 'bernoulli', n = n, p = p, mu = mu, stdev = stdev, signal_strength = signal_strength) 

In [18]:
df = pd.DataFrame(X)
y = np.array(y)
y = np.where(y>0,1,-1)
df['y'] = pd.Series(y)

In [19]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,y
0,0.035802,0.014548,0.000224,0.042836,0.012133,0.012024,-0.010876,0.006880,0.001727,-0.013595,...,0.019055,0.001349,-0.021834,-0.006255,0.016271,-0.003561,0.020491,-0.017084,-0.004861,-1
1,0.015747,0.001540,0.017395,-0.025157,-0.007037,0.044886,0.001102,-0.002860,0.006460,0.010718,...,0.001161,0.009108,-0.017133,-0.012412,-0.018023,0.007977,-0.002129,0.003515,-0.006680,-1
2,0.026607,0.011212,-0.017096,-0.009504,0.009121,-0.003865,0.003737,-0.017229,0.014224,-0.009546,...,-0.006778,0.000132,-0.021369,-0.009533,0.006321,-0.053178,-0.004010,-0.004128,0.020211,1
3,0.007634,-0.034306,0.005420,0.011555,0.007561,-0.007709,-0.010572,-0.010215,0.010504,0.007479,...,0.012683,0.015674,-0.000840,0.037685,-0.007669,0.004795,0.010116,-0.005950,-0.008575,-1
4,0.034167,-0.002704,0.000898,-0.007712,-0.005516,-0.007452,0.018969,-0.009643,0.010360,-0.006008,...,-0.020755,-0.004497,-0.017650,0.006590,-0.012821,0.003285,-0.010952,-0.024447,-0.013953,-1


In [20]:
X = df.iloc[:,:-1]
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.035802,0.014548,0.000224,0.042836,0.012133,0.012024,-0.010876,0.006880,0.001727,-0.013595,-0.005820,0.019055,0.001349,-0.021834,-0.006255,0.016271,-0.003561,0.020491,-0.017084,-0.004861
1,0.015747,0.001540,0.017395,-0.025157,-0.007037,0.044886,0.001102,-0.002860,0.006460,0.010718,-0.004170,0.001161,0.009108,-0.017133,-0.012412,-0.018023,0.007977,-0.002129,0.003515,-0.006680
2,0.026607,0.011212,-0.017096,-0.009504,0.009121,-0.003865,0.003737,-0.017229,0.014224,-0.009546,0.001929,-0.006778,0.000132,-0.021369,-0.009533,0.006321,-0.053178,-0.004010,-0.004128,0.020211
3,0.007634,-0.034306,0.005420,0.011555,0.007561,-0.007709,-0.010572,-0.010215,0.010504,0.007479,-0.026986,0.012683,0.015674,-0.000840,0.037685,-0.007669,0.004795,0.010116,-0.005950,-0.008575
4,0.034167,-0.002704,0.000898,-0.007712,-0.005516,-0.007452,0.018969,-0.009643,0.010360,-0.006008,0.014362,-0.020755,-0.004497,-0.017650,0.006590,-0.012821,0.003285,-0.010952,-0.024447,-0.013953


In [21]:
y = df.iloc[:,-1]
y

0      -1
1      -1
2       1
3      -1
4      -1
       ..
3995    1
3996    1
3997   -1
3998    1
3999   -1
Name: y, Length: 4000, dtype: int64

In [22]:
myX = [-2,-1,1,2]
X = pd.DataFrame(myX)
y = [-1,1,-1,1]

In [23]:
n,p = X.shape
print(n,p)

4 1


In [26]:
from gurobipy import *

m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]
m.update()


float32_epsilon = (np.finfo(np.float32).eps)*10 
            ### We wanted strict inequality, but Gurobi can not handle strict inequality
            # So, Had to use machine epsilon as reference. However, machine epsilon turned out to be too small
            # So, we used ten times machine epsilon
            # Machine epsilon can be considered as the smallest number greater than zero.
            # https://kite.com/python/answers/how-to-find-machine-epsilon-using-numpy-in-python

# Constraints
for i in range(n):
    m.addConstr(y[i]*(X.iloc[i].dot(v))>= float32_epsilon)
m.update()


# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model
if m.status == 3:
    model_status = 'Infeasible'
else:
    model_status = 'Optimal or Others'
print('Model Status:',model_status)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4 rows, 1 columns and 4 nonzeros
Model fingerprint: 0xd85ed01f
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds
Infeasible model
Model Status: Infeasible
